<a href="https://colab.research.google.com/github/EllouziMedAmin/Data-Science/blob/main/choufli_hal_pos_estimation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install cvzone

In [3]:
!pip install yt_dlp

In [10]:
from google.colab.patches import cv2_imshow

In [12]:
import cv2
from ultralytics import YOLO
import numpy as np
import cvzone

# Load the YOLO pose model
model = YOLO("yolo11n-pose.pt")

# Load the trimmed video
cap = cv2.VideoCapture('/content/trimmed_video.mp4')

# Set output video parameters
output_width, output_height = 1280, 720
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
out = cv2.VideoWriter('chouflihal.mp4', fourcc, fps, (output_width, output_height))

# Function to calculate the joint angle
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (output_width, output_height))
    results = model.predict(frame, save=False)

    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    keypoints_data = results[0].keypoints.data
    statuses = []

    for i, keypoints in enumerate(keypoints_data):
        if keypoints.shape[0] > 0:
            angle = calculate_angle(
                keypoints[11][:2].cpu().numpy(),
                keypoints[13][:2].cpu().numpy(),
                keypoints[15][:2].cpu().numpy()
            )
            status = 'Sitting' if angle < 110 else 'Standing'
            statuses.append(status)
        else:
            statuses.append("Unknown")

    # Draw results
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cvzone.putTextRect(
            frame, f"{statuses[i]}", (x1, y2 - 10),
            scale=2, thickness=2,
            colorT=(255, 255, 255), colorR=(255, 0, 255),
            font=cv2.FONT_HERSHEY_PLAIN,
            offset=10, border=0, colorB=(0, 255, 0)
        )

    # Draw pose directly from model
    pose_img = results[0].plot()
    out.write(pose_img)  # Save frame with predictions

# Release resources
cap.release()
out.release()
print("✅ Video saved as chouflihal.mp4")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Speed: 2.4ms preprocess, 18.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.5ms
Speed: 2.6ms preprocess, 11.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.6ms
Speed: 2.5ms preprocess, 10.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.0ms
Speed: 2.6ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 10.1ms
Speed: 2.6ms preprocess, 10.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 12.5ms
Speed: 2.1ms preprocess, 12.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.5ms
Speed: 2.2ms preprocess, 11.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 11.4ms
Speed: 2.1ms preprocess, 11.4ms inferenc

In [13]:
!ffmpeg -i chouflihal.mp4 -i trimmed_video.mp4 -c copy -map 0:v:0 -map 1:a:0 -shortest chouflihal_with_audio.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab